In [233]:
import glob
import os
import random
import tensorflow as tf

In [234]:
#declare global variables
TRAIN_SET_DIR = os.path.join('..', 'archive','TRAIN_V2','data_out_2')
RANDOM_SEED = 42
VALIDATION_RATE = 0.1
SAMPLING_RATE = 22050
BATCH_SIZE = 32

In [235]:
#create array to store all the training song paths
song_paths = []
song_genre_nums = []

# loop over each music genre
for genre_num in os.listdir(TRAIN_SET_DIR):
    
    # in each music genre, add the paths of all the music to the array
    for song_path in os.listdir(os.path.join(TRAIN_SET_DIR, genre_num)):
        song_paths.append(os.path.join(TRAIN_SET_DIR, genre_num, song_path))
        song_genre_nums.append(genre_num)

In [236]:
# suffle both arrays with the same shuffle seed
random.seed(RANDOM_SEED)
random.shuffle(song_paths)
random.seed(RANDOM_SEED)
random.shuffle(song_genre_nums)

In [237]:
# split both into training and testing sets
num_songs = len(song_genre_nums)
num_validation = int(num_songs * VALIDATION_RATE)

print("number of validations")
print(num_validation)
print('number of trainings')
print(num_songs - num_validation)

validation_genres = song_genre_nums[-num_validation :]
validation_paths = song_paths[-num_validation :]

training_genres = song_genre_nums[: -num_validation]
training_paths = song_paths[: -num_validation]

number of validations
1990
number of trainings
17919


In [238]:
# some useful funcs

# create tf dataset from the array of paths and flags
def create_audio_dataset(paths, genre_nums):
    paths = tf.data.Dataset.from_tensor_slices(paths);
    samples = paths.map(path_to_audio_samples)
    flags = tf.data.Dataset.from_tensor_slices(genre_nums)
    return tf.data.Dataset.zip((samples, flags))
    
# given an audio file path, return audio samples
def path_to_audio_samples(path):
    audio = tf.io.read_file(path)
    audio_tensor, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE)
    return audio_tensor

def audio_to_fft(audio_data):
    # call fft on first element, keep second element as is
    audio_sample = audio_data[0]

In [239]:
validation_dataset = create_audio_dataset(validation_paths, validation_genres)
training_dataset = create_audio_dataset(training_paths, training_genres)

In [240]:
validation_dataset.shuffle(RANDOM_SEED).batch(BATCH_SIZE)
training_dataset.shuffle(RANDOM_SEED).batch(BATCH_SIZE)

<BatchDataset shapes: ((None, 22050, 1), (None,)), types: (tf.float32, tf.string)>